## We have a 100x100 board.
### We throw 3 stones a Million times.
<img src="board.png" width="200">

### Given the XY position of each stone in the board <br> can we calculate how many of them forms a valid triangle?

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext.getOrCreate()

### we take string lines

In [4]:
triangles = sc.textFile("triangles.csv")
triangles.take(5)

['13 27, 68 55, 12 62',
 '64 96, 91 62, 43 51',
 '28 44, 93 59, 68 4',
 '60 8, 87 65, 93 98',
 '4 65, 2 77, 18 85']

<h1><center>Lambda Expressions</center> <img src="lambda.jpg" width="400" align="middle"/></h1>

In [6]:
distance = lambda p1, p2: ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
def sure_distance(p1, p2):
    x1, y1 = p1 # (13, 27)
    x2, y2 = p2 # (68, 55)
    dist = ((x2 - x1)**2 + (y2- y1)**2)**0.5
    return dist

In [7]:
distance((4,0), (0,3))
sure_distance((4,0), (0,3))

5.0

In [8]:
l = [(1,'z'), (2,'a')]

In [9]:
sorted(l)

[(1, 'z'), (2, 'a')]

In [10]:
sorted(l, key=lambda x: x[1])

[(2, 'a'), (1, 'z')]

### Let's deal with each line alone

In [14]:
sample_line = triangles.take(1)[0]
sample_line

'13 27, 68 55, 12 62'

### 1) split the string line by comma ,

In [15]:
split_comma = lambda line: line.split(',')
splitted_sample = split_comma(sample_line)
splitted_sample

['13 27', ' 68 55', ' 12 62']

### 2.1) Convert a string point to a tuple of integer x-y position

In [34]:
point_str2tuple = lambda point_str:(int(point_str.split()[0])  ## this is just a substep
                                       ,int(point_str.split()[1]))
point_str2tuple(splitted_sample[0])

(13, 27)

### 2.2) Take every string point in the list and convert it to x-y integer position

In [19]:
splitted2points = lambda splitted_line: [point_str2tuple(i) for i in splitted_line]
sample_points = splitted2points(splitted_sample)
sample_points

[(13, 27), (68, 55), (12, 62)]

### 3) Validate a triangle

In [29]:
def valid_triangle(p1, p2, p3):
    l1 = distance(p1, p2)
    l2 = distance(p2, p3)
    l3 = distance(p3, p1)
    return l1+l2>l3 and l1+l3>l2 and l2+l3 > l1

In [30]:
points_valid_triangle = lambda points: valid_triangle(*points)
points_valid_triangle(sample_points)

True

### 4) Combine all previous steps in one function

In [32]:
line2valid = lambda line: points_valid_triangle(
    splitted2points(
        split_comma(line)))
line2valid(sample_line)

True

<h1><center>Map/Reduce</center></h1>
<img src="mapreduce.jpeg" width="1000">

In [37]:
valid_triangles = triangles.map(line2valid)
valid_triangles.take(5)

[True, True, True, True, True]

In [38]:
num_triangles = valid_triangles.reduce(lambda x,y: x+y)
num_triangles

999074

![title](oneline.jpg)

In [41]:
num_triangles_short = triangles.map(lambda line:line.split(','))\
                                .map(lambda splitted_line: [(int(point_str.split()[0]),int(point_str.split()[1]))
                                        for point_str in splitted_line])\
                                .map(lambda points: valid_triangle(*points))\
                                .reduce(lambda x, y: x+y)
num_triangles_short

999074